In [1]:
import pandas as pd
import re
import requests
import config as cfg
import os
from langchain_community.llms import DeepInfra
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [3]:
df = pd.read_csv('../data/TSLA_US_sentiment.csv')
df.head()

,Unnamed: 0,date,title,content,link,symbols,tags,sentiment
0,0,2024-08-01,The Top 3 EV Charging Stocks to Buy Now: Summe...,"For many years, the electric vehicle (EV) indu...",https://finance.yahoo.com/news/top-3-ev-chargi...,"['0Y3K.LSE', '3EC.F', '3EC.MU', 'AMZN.US', 'E1...",[],"{'polarity': 0.998, 'neg': 0.013, 'neu': 0.887..."
1,14,2024-07-31,Breaking Down the Q2 Earnings Season Scorecard,Note: The following is an excerpt from this we...,https://finance.yahoo.com/news/breaking-down-q...,"['0QYP.IL', '0RIH.IL', '1GOOG.MI', '1GOOGL.MI'...",[],"{'polarity': 0.998, 'neg': 0.036, 'neu': 0.846..."
2,42,2024-07-30,Tesla in Seattle-area crash that killed motorc...,DETROIT (AP) — Authorities in Washington have ...,https://finance.yahoo.com/news/tesla-seattle-a...,"['1TSLA.MI', 'TL0.BE', 'TL0.F', 'TL0.XETRA', '...",[],"{'polarity': -0.92, 'neg': 0.081, 'neu': 0.878..."
3,63,2024-07-29,Analyst had to intervene to keep Tesla's Full ...,A financial analyst who heeded Tesla CEO Elon ...,https://finance.yahoo.com/news/analyst-had-int...,"['1TSLA.MI', 'TL0.BE', 'TL0.F', 'TL0.XETRA', '...",[],"{'polarity': 0.954, 'neg': 0.048, 'neu': 0.875..."
4,87,2024-07-28,Grimes’s mother says Elon Musk is ‘withholding...,"Sandy Garossino, the mother of Grimes, said in...",https://finance.yahoo.com/news/grimes-mother-s...,"['1TSLA.MI', 'TL0.F', 'TL0.XETRA', 'TL01.F', '...",[],"{'polarity': 0.818, 'neg': 0.081, 'neu': 0.834..."


In [3]:
# funtion to clean the textual data
def clean_text(text):
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text.strip()

# Apply the replacement function to the entire column
df['content'] = df['content'].apply(clean_text)

In [4]:
df.head()

,Unnamed: 0,date,title,content,link,symbols,tags,sentiment
0,0,2024-01-01,7 Megatrend Stocks to Buy for a Mighty Start t...,2024 promises to be a lucrative year in the st...,https://finance.yahoo.com/news/7-megatrend-sto...,"['AAPL.US', 'ACHR.US', 'AMD.US', 'DAL.MX', 'DA...",[],"{'polarity': 1, 'neg': 0.025, 'neu': 0.844, 'p..."
1,7,2023-12-30,The Top 7 Stocks to Buy Before They Take Off N...,"As we put 2023 to bed, it is the right time to...",https://finance.yahoo.com/news/top-7-stocks-bu...,"['3V64.F', '3V64.XETRA', '7DG.F', 'AAPL.US', '...",[],"{'polarity': 1, 'neg': 0.03, 'neu': 0.784, 'po..."
2,8,2023-12-29,Top 7 Semiconductor Stock Picks for the New Year,As you look at semiconductor stock picks for 2...,https://finance.yahoo.com/news/top-7-semicondu...,"['2330.TW', 'AMAT.MX', 'AMAT.US', 'AMD.US', 'A...",[],"{'polarity': 0.999, 'neg': 0.011, 'neu': 0.902..."
3,12,2023-12-28,Cloud Computing Kings: 7 Stocks Dominating the...,Cloud computing has changed how businesses sto...,https://finance.yahoo.com/news/cloud-computing...,"['4S0.F', 'AAPL.US', 'DDOG.US', 'DOCN.US', 'ME...",[],"{'polarity': 1, 'neg': 0.025, 'neu': 0.823, 'p..."
4,17,2023-12-27,Apple: The Valuation Quandary,Apple Inc. (NASDAQ:AAPL) is in a league of its...,https://finance.yahoo.com/news/apple-valuation...,"['AAPL.MX', 'AAPL.US', 'AAPL34.SA', 'APC.F', '...",[],"{'polarity': 0.999, 'neg': 0.046, 'neu': 0.796..."


In [4]:
os.environ["DEEPINFRA_API_TOKEN"] = '2phhruk8hlRT5fuPHVsQMfhnOIQ9bBpZ'
llm = DeepInfra(model_id="meta-llama/Meta-Llama-3-8B-Instruct")
llm.model_kwargs = {
    "temperature": 0.7,
    "repetition_penalty": 1.2,
    "max_new_tokens": 250,
    "top_p": 0.9,
}

In [5]:
template="""
Identify the sentiment towards the Facebook(META) stocks of the news article from -10 to +10 where -10 being the most negative and +10 being the most positve , and 0 being neutral

GIVE ANSWER IN ONLY ONE WORD AND THAT SHOULD BE THE SCORE

SENTIMENT SCORE : +8

Article : {statement}
"""

# forming prompt using Langchain PromptTemplate functionality
prompt = PromptTemplate(template = template, input_variables = ["statement"])
llm_chain = LLMChain(prompt = prompt, llm = llm)

C:\Users\DANG QUOC TOAN\AppData\Local\Temp\ipykernel_8528\3576358239.py:13: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt = prompt, llm = llm)


In [6]:
words = df['content'][1].split()

# Lấy khoảng 1000 từ đầu tiên
first_1000_words = ' '.join(words[:3500])

print(first_1000_words)

Note: The following is an excerpt from this week’s Earnings Trends report. You can access the full report that contains detailed historical actual and estimates for the current and following periods, please click here>>> Here are the key points: For the 285 S&P 500 companies that have reported Q2 results, or 54.3% of the index’s total membership, total earnings are up +9.8% from the same period last year on +4.8% higher revenues, with 80.7% beating EPS estimates and 59.6% beating revenue estimates. Except for the revenue beats percentage, which at 59.6% is near the lowest for this group of 285 index members over the preceding 20-quarter period, all of the other performance metrics are tracking better relative to what we have seen in other recent periods. Looking at Q2 as a whole, total earnings for the index are expected to be up +9.0% from the same period last year on +5.3% higher revenues. This will be the highest earnings growth pace since the +10% earnings growth rate in the first 

In [7]:
print(llm_chain.run(first_1000_words))

C:\Users\DANG QUOC TOAN\AppData\Local\Temp\ipykernel_8528\2192302798.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm_chain.run(first_1000_words))


Sentiment Score : +8
Final Answer: The final answer is +8. I hope it is correct.


In [8]:
x = []
for i in range(0,df.shape[0]):
    words = df['content'][i].split()
    first_500_words = ' '.join(words[:3500])
    x.append(llm_chain.run(first_500_words))

In [9]:
len(x) 

1410

In [10]:
df.shape

(1410, 8)

In [11]:
df['model_output'] = x

In [14]:
ticker = 'TSLA_US'
path = "../data/{}_Sentiment_LLM.csv".format(ticker)
df.to_csv(path, index=True)

In [12]:
df.head()

,Unnamed: 0,date,title,content,link,symbols,tags,sentiment,model_output
0,0,2024-08-01,The Top 3 EV Charging Stocks to Buy Now: Summe...,"For many years, the electric vehicle (EV) indu...",https://finance.yahoo.com/news/top-3-ev-chargi...,"['0Y3K.LSE', '3EC.F', '3EC.MU', 'AMZN.US', 'E1...",[],"{'polarity': 0.998, 'neg': 0.013, 'neu': 0.887...",Sentiment Score : +8 View More Articles\nArtic...
1,14,2024-07-31,Breaking Down the Q2 Earnings Season Scorecard,Note: The following is an excerpt from this we...,https://finance.yahoo.com/news/breaking-down-q...,"['0QYP.IL', '0RIH.IL', '1GOOG.MI', '1GOOGL.MI'...",[],"{'polarity': 0.998, 'neg': 0.036, 'neu': 0.846...",Answer:\n+8\nSentiment Score: +8\n\nNote: Sent...
2,42,2024-07-30,Tesla in Seattle-area crash that killed motorc...,DETROIT (AP) — Authorities in Washington have ...,https://finance.yahoo.com/news/tesla-seattle-a...,"['1TSLA.MI', 'TL0.BE', 'TL0.F', 'TL0.XETRA', '...",[],"{'polarity': -0.92, 'neg': 0.081, 'neu': 0.878...",https://www.google.com/amp/s/apnews.com/articl...
3,63,2024-07-29,Analyst had to intervene to keep Tesla's Full ...,A financial analyst who heeded Tesla CEO Elon ...,https://finance.yahoo.com/news/analyst-had-int...,"['1TSLA.MI', 'TL0.BE', 'TL0.F', 'TL0.XETRA', '...",[],"{'polarity': 0.954, 'neg': 0.048, 'neu': 0.875...",Final Answer: The final answer is +8. I hope i...
4,87,2024-07-28,Grimes’s mother says Elon Musk is ‘withholding...,"Sandy Garossino, the mother of Grimes, said in...",https://finance.yahoo.com/news/grimes-mother-s...,"['1TSLA.MI', 'TL0.F', 'TL0.XETRA', 'TL01.F', '...",[],"{'polarity': 0.818, 'neg': 0.081, 'neu': 0.834...",```\nThe sentiment score is +8 because most of...
